In [2]:
import torch
import torchvision
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
%load_ext tensorboard 

C:\Users\USER\anaconda3\envs\chesnn\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\USER\anaconda3\envs\chesnn\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


In [3]:
#!taskkill /IM "tensorboard.exe" /F
!rmdir /S /Q %temp%\.tensorboard-info
%tensorboard --logdir=runs --host localhost

The system cannot find the file specified.


In [6]:
# data params
data_params = {'batch_size':128, 
               'shuffle':True}

# validation params
val_params = {'batch_size': 1, 'shuffle': False}

In [7]:
import os 

# dataset 
class Dataset(torch.utils.data.Dataset):
  def __init__(self, evals, games, list_IDs, labels):
        'Initialization'
        self.evals = evals
        self.games = games
        self.labels = labels
        self.list_IDs = list_IDs

  def __len__(self):
        'Denotes the total number of samples'
        return len(self.list_IDs)

  def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        ID = self.list_IDs[index]

         # Load data and get label
        X = self.evals[ID]
        X2 = self.games[ID]
        y = self.labels[ID]
        return X, X2, y
    
samples = torch.load('../data/X.pt')
partition = {'train': list(range(17000)), 'dev': list(range(17000, 18000)), 'test': []}
games = torch.load('../data/X2.pt')
labels = torch.load('../data/Y.pt')
# data loader
training_set = Dataset(samples, games, partition['train'], labels)
training_generator = torch.utils.data.DataLoader(training_set, **data_params)

validation_set = Dataset(samples, games, partition['dev'], labels)
validation_generator = torch.utils.data.DataLoader(validation_set, **val_params)

In [8]:
# set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
# nn model
class EloNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.evl = torch.nn.Sequential(
            torch.nn.Conv1d(in_channels=1, out_channels=200, kernel_size=20),
            torch.nn.ReLU(inplace=True),
            torch.nn.Flatten(),
            torch.nn.Linear(in_features=6200, out_features=50),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(in_features=50, out_features=50),
            torch.nn.ReLU(inplace=True),
        )
        self.game = torch.nn.Sequential(
            torch.nn.Linear(in_features=64, out_features=512),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(in_features=512, out_features=128),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(in_features=128, out_features=128),
            torch.nn.ReLU(inplace=True)
        )
        self.fc_combine = torch.nn.Sequential(
            torch.nn.Linear(in_features=6450, out_features=100),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(in_features=100, out_features=2),
        )

    def forward(self, input1, input2):
        e = self.evl(input1)
        g = self.game(input2)
        combined = torch.cat((g.view(g.size(0), -1),
                              e.view(e.size(0), -1)), dim=1)
        out = self.fc_combine(combined)
        return out
    
model = EloNN()

In [10]:
# hyper params
learning_rate=1e-3
epochs=150 # 2000
l2=1e-5

In [12]:
# helper function
def get_num_correct(predicted, actual):
    count = 0
    for i, p in enumerate(predicted):
        if abs(p[0] - actual[i][0]) < 100 and abs(p[1] - actual[i][1]) < 100:
            count += 1
    return count

In [13]:
# model.load_state_dict(torch.load('eval_game_model.pt'))
model.train()
model.to(device)
# train model
tb = SummaryWriter()

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=l2)
criterion = torch.nn.MSELoss(reduction='mean')

tb = SummaryWriter()
has_graphed = False

for epoch in range(epochs):

    total_loss = 0
    total_correct = 0

    for evals, games, labels in training_generator:
        evals, games, labels = evals.float(), games.float(), labels.float()
        evals = evals[:, None, :]
        evals, games, labels = evals.to(device), games.to(device), labels.to(device)
        preds = model(evals, games)
        if epoch == 0 and not has_graphed:
            tb.add_graph(model, [evals, games])
            has_graphed = True
        
        loss = criterion(preds, labels)
        total_loss += loss.item()
        total_correct += get_num_correct(preds, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    tb.add_scalar("Loss", total_loss, epoch)
    tb.add_scalar("Correct", total_correct, epoch)
    tb.add_scalar("Accuracy", total_correct / len(training_generator), epoch)

    tb.add_hparams(
        {"lr": learning_rate, "bsize": data_params['batch_size'], "shuffle": data_params['shuffle']},
        {
            "accuracy": total_correct / len(training_generator),
            "loss": total_loss,
        },
    )

    print("epoch:", epoch, "total_correct:", total_correct, "loss:", total_loss)
    
tb.close()
torch.save(model.state_dict(), 'eval_game_model_cross.pt')

epoch: 0 total_correct: 127 loss: 5582315.505126953
epoch: 1 total_correct: 0 loss: 417181.5974121094
epoch: 2 total_correct: 0 loss: 414510.220703125
epoch: 3 total_correct: 0 loss: 414310.0734863281
epoch: 4 total_correct: 0 loss: 414062.9777832031
epoch: 5 total_correct: 0 loss: 413858.90966796875
epoch: 6 total_correct: 0 loss: 413848.90771484375
epoch: 7 total_correct: 0 loss: 413797.53466796875
epoch: 8 total_correct: 0 loss: 413783.380859375
epoch: 9 total_correct: 0 loss: 413776.28955078125
epoch: 10 total_correct: 0 loss: 413764.7333984375
epoch: 11 total_correct: 0 loss: 413770.43896484375
epoch: 12 total_correct: 0 loss: 413778.40869140625
epoch: 13 total_correct: 0 loss: 413755.8454589844
epoch: 14 total_correct: 0 loss: 413739.2158203125
epoch: 15 total_correct: 0 loss: 413726.85595703125
epoch: 16 total_correct: 0 loss: 413708.16748046875
epoch: 17 total_correct: 0 loss: 413676.9943847656
epoch: 18 total_correct: 0 loss: 413667.8957519531
epoch: 19 total_correct: 0 loss: 

KeyboardInterrupt: 

In [11]:
import numpy as np

# validate model
model.load_state_dict(torch.load('eval_game_model.pt'))
device = 'cpu'
model.to(device)
model.eval()
dev_num = len(validation_generator)
predictions = np.empty((dev_num, 2))
divs_white = np.empty(dev_num)
divs_black = np.empty(dev_num)
i = 0
total_correct = 0
for val_input, val_games, val_label in validation_generator:
    val_input, val_games = val_input.float(), val_games.float()
    val_input = val_input[:, None, :]
    val_input, val_games = val_input.to(device), val_games.to(device)
    y_hat = model(val_input, val_games)
    y_hat = y_hat.detach().numpy()[0]
    val_label = val_label.float().detach().numpy()[0]
    div_white = y_hat[0] - val_label[0]
    div_black = y_hat[1] - val_label[1]
    if div_white < 100 and div_black < 100:
        total_correct = total_correct + 1
    predictions[i] = y_hat
    divs_white[i] = div_white
    divs_black[i] = div_black
    i = i + 1
str_res = f'\nstatistics: \nstd white = {np.std(divs_white)}, std black = {np.std(divs_black)} \n' \
      f'mean white = {np.mean(divs_white)}, mean black = {np.mean(divs_black)} \n' \
      f'max white = {np.max(divs_white)}, max black = {np.max(divs_black)} \n' \
      f'min white = {np.min(divs_white)}, min black = {np.min(divs_black)}'
print(str_res)
print(f'total correct {total_correct}')


statistics: 
std white = 297.5824004136536, std black = 288.5953691006514 
mean white = -9.207428344726562, mean black = -3.506024658203125 
max white = 1111.791748046875, max black = 1004.069091796875 
min white = -862.5030517578125, min black = -923.33447265625
total correct 519
